In [183]:
# Initialize stage
import numpy as np

n_stages = 25
stages = np.zeros([5, 5])
print(stages)

# Set action
n_action = 4
action = ['U', 'D', 'L', 'R']

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [184]:
GOAL1 = (0, 1)
GOAL2 = (0, 3)

def env(s, a):
    flag = 0
    s_org = s
    row = s[0]
    col = s[1]
    if a == 'U':
        row -= 1
    elif a == 'D':
        row += 1
    elif a == 'L':
        col -= 1
    elif a == 'R':
        col += 1
    else:
        raise Exception("Invalid action '" + a + "'")
    
    # Go out of grid in north directions
    if row < 0:
        flag = -1
        row = 0
    
    # South
    if row >= ROWS:
        flag = -1
        row = ROWS-1
    
    # East
    if col < 0:
        flag = -1
        col = 0
    
    # West
    if col >= COLS:
        flag = -1
        col = COLS-1

    s = (row, col)

    
    if (s_org == GOAL1) or (s_org == GOAL2):
        return s, 0, False

    if s == GOAL1:
        return s, 10.0, True # Warp agent to A prime
    elif s == GOAL2:
        return s, 5.0, True # Warp agent to B Prime
    

    
    return s, flag, False

In [185]:
ROWS = 5
COLS  = 5
epsilon = 0.0001
gamma = 0.9
prob = 0.25
temp_diff = 1
counter = 0

In [186]:
def value_iteration(env, gamma, threshold):
    n_state = 25
    n_action = 4
    V = np.zeros(n_state)
    while True:
        V_temp = np.empty(n_state)
        for state in range(n_state):
            V_actions = np.zeros(n_action)
            for action in range(n_action):
                for prob, next_state, reward, terminal in env.env.P[state][action]:
                    V_actions[action] += prob * (reward + gamma * V[next_state])
            V_temp[state] = np.max(V_actions)
        max_delta = np.max(np.abs(V - V_temp))
        V = V_temp.copy()
        if max_delta <= threshold:
            break
    return V

In [187]:
# Set rewards
terminate_zone = [(0, 1), (0, 3)]
while True:
    for curr_row in range(ROWS): # Get index of row
        for curr_col in range(COLS): # Get index of column
            V_actions = np.zeros(n_action) # Set temporary variables
            for idx, curr_actions in enumerate(action):
                curr_stages = (curr_row, curr_col)
                next_stage, reward, terminate = env(curr_stages, curr_actions)

                next_stage_row = next_stage[0]
                next_stage_col = next_stage[1]

                if terminate == True:
                    gammav = 0
                else:
                    gammav = stages[next_stage_row][next_stage_col]
                    
                V_actions[idx] += prob * (reward + (gamma * gammav))
            
            if (curr_row, curr_col) in terminate_zone:
                    copy_stages[curr_row][curr_col] = 0
            else:
                copy_stages[curr_row][curr_col] = np.max(V_actions)

                
    
    # Calculate temporal difference
    max_delta = np.max(np.abs(stages - copy_stages))
    stages = np.copy(copy_stages)
    
    # Stop by the values of epsilon and counter
    if max_delta <= epsilon or counter > 1000000:
        print(f'force stop temp_diff = {max_delta}, counter = {counter}')
        break
    
    
    counter += 1

                
                

force stop temp_diff = 0.0, counter = 5


In [188]:
# Value at each stages.
stages

array([[2.5       , 0.        , 2.5       , 0.        , 1.25      ],
       [0.5625    , 2.5       , 0.5625    , 1.25      , 0.28125   ],
       [0.1265625 , 0.5625    , 0.1265625 , 0.28125   , 0.06328125],
       [0.02847656, 0.1265625 , 0.02847656, 0.06328125, 0.01423828],
       [0.00640723, 0.02847656, 0.00640723, 0.01423828, 0.00320361]])

In [213]:
# Action - value functions
action_val = list()
terminate_zone = [(0, 1), (0, 3)]

for curr_row in range(ROWS): # Get index of row
    for curr_col in range(COLS): # Get index of column
        V_actions = np.zeros(n_action) # Set temporary variables
        temp_list = list()
        for idx, curr_actions in enumerate(action):
            curr_stages = (curr_row, curr_col)
            
            #action_val[curr_stages] = {}
            next_stage, reward, terminate = env(curr_stages, curr_actions)

            next_stage_row = next_stage[0]
            next_stage_col = next_stage[1]

            if terminate == True:
                gammav = 0
            else:
                gammav = stages[next_stage_row][next_stage_col]

            V_actions[idx] += prob * (reward + (gamma * gammav))
            
            if curr_stages in terminate_zone:
                temp_list.append(0)
            else:
                temp_list.append({curr_actions : prob * (reward + (gamma * gammav))})
        action_val.append({(curr_row, curr_col): temp_list})
    

In [214]:
# Action-value
action_val

[{(0, 0): [{'U': 0.3125}, {'D': 0.1265625}, {'L': 0.3125}, {'R': 2.5}]},
 {(0, 1): [0, 0, 0, 0]},
 {(0, 2): [{'U': 0.3125}, {'D': 0.1265625}, {'L': 2.5}, {'R': 1.25}]},
 {(0, 3): [0, 0, 0, 0]},
 {(0, 4): [{'U': 0.03125}, {'D': 0.06328125}, {'L': 1.25}, {'R': 0.03125}]},
 {(1, 0): [{'U': 0.5625},
   {'D': 0.0284765625},
   {'L': -0.1234375},
   {'R': 0.5625}]},
 {(1, 1): [{'U': 2.5}, {'D': 0.1265625}, {'L': 0.1265625}, {'R': 0.1265625}]},
 {(1, 2): [{'U': 0.5625}, {'D': 0.0284765625}, {'L': 0.5625}, {'R': 0.28125}]},
 {(1, 3): [{'U': 1.25},
   {'D': 0.06328125},
   {'L': 0.1265625},
   {'R': 0.06328125}]},
 {(1, 4): [{'U': 0.28125},
   {'D': 0.01423828125},
   {'L': 0.28125},
   {'R': -0.18671875}]},
 {(2, 0): [{'U': 0.1265625},
   {'D': 0.0064072265625},
   {'L': -0.2215234375},
   {'R': 0.1265625}]},
 {(2, 1): [{'U': 0.5625},
   {'D': 0.0284765625},
   {'L': 0.0284765625},
   {'R': 0.0284765625}]},
 {(2, 2): [{'U': 0.1265625},
   {'D': 0.0064072265625},
   {'L': 0.1265625},
   {'R': 0